In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams, clfName, distanceMetric, beta=None):
    if distanceMetric == 'BBD' and isinstance(beta,list):
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_entireData_{}_EVL-{}-ALLBETAS.txt".format(distanceMetric,clfName)
    elif distanceMetric == 'BBD':
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_entireData_{}{:.3f}_EVL-{}.txt".format(distanceMetric, beta,clfName)
    else:
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_entireData_{}_EVL-{}.txt".format(distanceMetric,clfName)
        
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadNOAADataset, setup.loadKeystroke]
    batches=100
    #datasets=[setup.loadKeystroke]
    #batches=8
    #datasets=[setup.loadNOAADataset]
    #batches=50
    
    arrClfName = ['LP']#['SGD', 'NB', 'RF', 'LP', 'KNN']
    distanceMetric = 'BBD'
    beta = [-50,-10, -5, -2, -1, -0.5, -0.1, -0.01, -0.001, 1.001, 1.01, 1.1, 1.5, 2, 3, 5, 10, 50]
    allBetas=True
    for clfName in arrClfName:
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("**************** BEGIN of {}-{}{:.3f} results ****************".format(clfName, distanceMetric, beta))            
        else:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        poolSize = None
        isBatchMode = True
        #testing grid search
        for i in range(len(datasets)):
            batches=100
            if i==len(datasets)-2:
                batches=50
            elif i==len(datasets)-1:
                batches=8
                
            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)
            
            initialLabeledData = int(0.05*len(dataLabels))
            sizeOfBatch = int((len(dataLabels)-initialLabeledData)/batches)

            print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

            tuned_params = [{"sizeOfBatch":[sizeOfBatch], "batches":[batches], "poolSize":[poolSize],
                             "isBatchMode":[isBatchMode], "initialLabeledData":[initialLabeledData], "clfName":[clfName],
                            "distanceMetric":[distanceMetric], "beta":beta}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_dynamic.run(**g)

                try:
                    gs.fit(dataValues, dataLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except Exception:
                    print("An error occured in ", description, g)
                    #raise Exception

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            writeResults(description, finalScore, best_grid, clfName, distanceMetric, beta)
        
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("******** END of {}-{}{:.3f} results ********".format(clfName, distanceMetric, beta))
        else:
            print("******** END of {}-{} results ********".format(clfName, distanceMetric))
    
if __name__ == "__main__":
    main()


**************** BEGIN of LP-BBD results ****************
One Class Diagonal Translation. 2 Dimensional data.: 100 batches of 152 instances


C:\Users\bruno.silva\Desktop\TCC\AMANDA\setup.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  dataValues = pd.DataFrame.as_matrix(dataValues)


99.94060000000002 {'K': 2, 'batches': 100, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.94720000000001 {'K': 2, 'batches': 100, 'beta': -10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.93400000000001 {'K': 2, 'batches': 100, 'beta': -5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.92750000000002 {'K': 2, 'batches': 100, 'beta': -2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.95380000000002 {'K': 2, 'batches': 100, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.95380000000002 {'K': 2, 'batches': 100, 'beta': -0.5, 'clfName': 'LP', 'distance

99.94060000000002 {'K': 4, 'batches': 100, 'beta': 1.001, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.94060000000002 {'K': 4, 'batches': 100, 'beta': 1.01, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.94720000000001 {'K': 4, 'batches': 100, 'beta': 1.1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.94060000000002 {'K': 4, 'batches': 100, 'beta': 1.5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.95380000000002 {'K': 4, 'batches': 100, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.94720000000001 {'K': 4, 'batches': 100, 'beta': 3, 'clfName': 'LP', 'distanc

99.94069999999999 {'K': 7, 'batches': 100, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.94069999999999 {'K': 7, 'batches': 100, 'beta': -10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.94069999999999 {'K': 7, 'batches': 100, 'beta': -5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.94069999999999 {'K': 7, 'batches': 100, 'beta': -2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.9341 {'K': 7, 'batches': 100, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.95389999999999 {'K': 7, 'batches': 100, 'beta': -0.5, 'clfName': 'LP', 'distanceMetric': '

...

65.12489999999998 {'K': 12, 'batches': 100, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
73.1177 {'K': 12, 'batches': 100, 'beta': -0.5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
60.131699999999995 {'K': 12, 'batches': 100, 'beta': -0.1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
51.4338 {'K': 12, 'batches': 100, 'beta': -0.01, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
51.025999999999996 {'K': 12, 'batches': 100, 'beta': -0.001, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
86.22989999999999 {'K': 12, 'batches': 100, 'beta': 1.001, 'clfName': 'LP', 'distanceMe

C:\Users\bruno.silva\Desktop\TCC\AMANDA\setup.py:176: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  dataValues = pd.DataFrame.as_matrix(dataValues)


99.98290000000001 {'K': 2, 'batches': 100, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1371}
99.98290000000001 {'K': 2, 'batches': 100, 'beta': -10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1371}
99.98290000000001 {'K': 2, 'batches': 100, 'beta': -5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1371}
99.9858 {'K': 2, 'batches': 100, 'beta': -2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1371}
99.9851 {'K': 2, 'batches': 100, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1371}
99.98360000000002 {'K': 2, 'batches': 100, 'beta': -0.5, 'clfName': 'LP', 'distanceMetric': '

99.9822 {'K': 4, 'batches': 100, 'beta': 1.001, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1371}
99.9822 {'K': 4, 'batches': 100, 'beta': 1.01, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1371}
99.9822 {'K': 4, 'batches': 100, 'beta': 1.1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1371}
99.9822 {'K': 4, 'batches': 100, 'beta': 1.5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1371}
99.98370000000001 {'K': 4, 'batches': 100, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1371}
99.98370000000001 {'K': 4, 'batches': 100, 'beta': 3, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabel

KeyboardInterrupt: 